### Step 2. 데이터 읽어오기

In [1]:
import re               
import numpy as np         
import tensorflow as tf 

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.', 'It was too wet to play.', 'So we sat in the house', 'All that cold cold wet day.', 'I sat there with Sally.', 'We sat there we two.', 'And I said How I wish', 'We had something to do!']


### Step 3. 데이터 정제

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence.split()) >=13 : continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  833 ...    0    0    0]
 [   2  118 2605 ...    0    0    0]
 [   2    6  297 ...    0    0    0]
 ...
 [   2 2513   13 ...    0    0    0]
 [   2  692   27 ...    0    0    0]
 [   2  692   27 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f7093b91d50>


### Step 4. 평가 데이터셋 분리

In [6]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2    6  833   14    6 1213    3    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[   6  833   14    6 1213    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [7]:
print(src_input.shape)
print(tgt_input.shape)

(158876, 32)
(158876, 32)


In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, 
                                                          random_state=2020, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127100, 32)
Target Train: (127100, 32)


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

In [11]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

### Step 5. 인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!     
(Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[-7.45320285e-05, -1.30433727e-05, -2.65727635e-04, ...,
         -1.99578226e-05, -9.88663305e-05,  3.91473441e-06],
        [-3.86386200e-05,  1.71864784e-04, -4.90912294e-04, ...,
         -1.56890033e-04,  1.45705882e-04,  1.35427937e-04],
        [ 2.15654407e-04,  3.73609917e-04, -6.43857929e-04, ...,
         -3.18878039e-04,  3.79037840e-04,  2.95564532e-04],
        ...,
        [-1.12380926e-03,  3.05116735e-03,  4.11078567e-03, ...,
         -1.24586129e-03,  3.66896647e-03,  3.59315658e-03],
        [-1.13702659e-03,  3.09197581e-03,  4.12643002e-03, ...,
         -1.28007005e-03,  3.68653447e-03,  3.62535543e-03],
        [-1.14945823e-03,  3.12704593e-03,  4.13766829e-03, ...,
         -1.30813406e-03,  3.69971246e-03,  3.65092745e-03]],

       [[-7.45320285e-05, -1.30433727e-05, -2.65727635e-04, ...,
         -1.99578226e-05, -9.88663305e-05,  3.91473441e-06],
        [-3.51632363e-04,  1.13287839e-04, -3

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

model.fit(dataset, validation_data=val_dataset, epochs=10)


Epoch 1/10
496/496 [==============================] - 144s 290ms/step - loss: 1.6379 - accuracy: 0.7676 - val_loss: 1.4050 - val_accuracy: 0.7817
Epoch 2/10
496/496 [==============================] - 146s 294ms/step - loss: 1.3512 - accuracy: 0.7859 - val_loss: 1.3199 - val_accuracy: 0.7882
Epoch 3/10
496/496 [==============================] - 159s 320ms/step - loss: 1.2771 - accuracy: 0.7909 - val_loss: 1.2700 - val_accuracy: 0.7914
Epoch 4/10
496/496 [==============================] - 169s 340ms/step - loss: 1.2212 - accuracy: 0.7945 - val_loss: 1.2344 - val_accuracy: 0.7945
Epoch 5/10
496/496 [==============================] - 161s 324ms/step - loss: 1.1728 - accuracy: 0.7979 - val_loss: 1.2049 - val_accuracy: 0.7975
Epoch 6/10
496/496 [==============================] - 177s 357ms/step - loss: 1.1278 - accuracy: 0.8015 - val_loss: 1.1805 - val_accuracy: 0.8001
Epoch 7/10
496/496 [==============================] - 160s 323ms/step - loss: 1.0859 - accuracy: 0.8051 - val_loss: 1.1584 -

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i will", max_len=20)

'<start> i will be your satellite <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> hi", max_len=20)

'<start> hi ! my name is huh <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> goodbye", max_len=20)

'<start> goodbye to the moon <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> i have", max_len=20)

'<start> i have a little bit of my heart <end> '

- 최종 val_loss: 1.1153 - val_accuracy: 0.8100    

tokenize함수를 만들때    
tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') 부분에 maxlen = 15로 설정하였다가,    
loss값이 잘 떨어지지않고, accuracy 도 0.5로 높지 않아서, maxlen 에 공부하게 되었다.     
manlen은 정해준 부분 이상이 되면 그 뒷 정보를 아예 삭제한다고 하여, 온전한 문장이 아닌, 설정한 '15'이후 단어를 자른 문장을 만든다고 한다.   
그래서 온전하지 않은 문장을 학습시켜서 정확도를 높이고, 로스를 낮추려고 하니 힘들었던것...    
그래서 corpus에 넣을때 아예 긴문장을 넣지않은 방법으로 하니, 노드에서 준 모델을 파라미터 변경없이 그대로 써도,     
accuracy가 0.81로 높게 나오고, loss도 평가기준인 2.2보다 낮은 1.1153이 나왔다.    